In [1]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser

prompt = ChatPromptTemplate.from_template(
    "tell me about {topic}"
)
model = ChatOpenAI()
output_parser = StrOutputParser()

chain = prompt | model | output_parser

chain.invoke({"topic": "dog"})

'Dogs are domesticated mammals that have been bred by humans for thousands of years. They are known for their loyalty, intelligence, and companionship. There are hundreds of different breeds of dogs, ranging in size, shape, and temperament.\n\nDogs are social animals that thrive on human interaction and companionship. They are known for their ability to form strong bonds with their owners and are often referred to as "man\'s best friend." Dogs are also highly trainable and can be taught a variety of commands and tricks.\n\nIn addition to being great companions, dogs also serve a variety of purposes including as service animals, therapy animals, search and rescue dogs, and working dogs in fields such as law enforcement and agriculture.\n\nOverall, dogs are beloved pets for many people around the world and are considered to be a valuable part of our lives.'

In [5]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import DocArrayInMemorySearch   

vectorstore = DocArrayInMemorySearch.from_texts(
    ["GGenerative AI is a type of artificial intelligence that creates new content, such as text, images, or code, based on learned patterns.","(LCEL)LangChain Expression Language (LCEL) is a declarative syntax for composing and chaining LangChain components efficiently."],
    embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()

retriever.get_relevant_documents("what is generative ai?")


[Document(page_content='GGenerative AI is a type of artificial intelligence that creates new content, such as text, images, or code, based on learned patterns.'),
 Document(page_content='(LCEL)LangChain Expression Language (LCEL) is a declarative syntax for composing and chaining LangChain components efficiently.')]

In [6]:
retriever.get_relevant_documents("what is the full form of LCEL")

[Document(page_content='(LCEL)LangChain Expression Language (LCEL) is a declarative syntax for composing and chaining LangChain components efficiently.'),
 Document(page_content='GGenerative AI is a type of artificial intelligence that creates new content, such as text, images, or code, based on learned patterns.')]

In [3]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

from langchain.schema.runnable import RunnableMap

chain = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
}) | prompt | model | output_parser

chain.invoke({"question": "what is the full form of LCEL?"})

'LangChain Expression Language'

In [4]:
inputs = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
})

inputs.invoke({"question": "what is the full form of LCEL?"})

{'context': [Document(page_content='(LCEL)LangChain Expression Language (LCEL) is a declarative syntax for composing and chaining LangChain components efficiently.'),
  Document(page_content='GGenerative AI is a type of artificial intelligence that creates new content, such as text, images, or code, based on learned patterns.')],
 'question': 'what is the full form of LCEL?'}